# Convert YOLO to tflite 
* 안드로이드에 YOLO 모델을 돌리기 위해서는 tensorflow 모델로 올려야함
* [tensorflow-yolov4-tflite](https://github.com/hunglc007/tensorflow-yolov4-tflite) : Darknet으로 생성한 yolo weight를 tflite 로 변환하기 위해 사용
* MIT License

## weight 로딩

### 구글 드라이브와 연동

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### 소프트링크 생성
* 편의성을 위함

In [ ]:
!ln -s /content/gdrive/My\ Drive/Colab\ Notebooks/CPDS /mydrive

### weight, obj.names 로딩
* weight : YOLO weight 파일
* obj.names : model의 classes 정보

In [ ]:
!cp /mydrive/yolov4-obj_best.weights model.weights
!cp /mydrive/obj.names obj.names

## git clone

In [ ]:
%cd /content
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite.git

/content
Cloning into 'tensorflow-yolov4-tflite'...
remote: Enumerating objects: 780, done.
remote: Total 780 (delta 0), reused 0 (delta 0), pack-reused 780
Receiving objects: 100% (780/780), 197.64 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (371/371), done.


## Convert


* save_model.py 를 실행하면 output_path에 pb 파일이 생성됨
* 생성된 pb파일의 경로가 다음에 실행할 conver_tflite.py의 weights의 경로가 됨
* tensorflow-yolov4-tflite/core/config.py 파일

### 1. config.py 수정
* /tensorflow-yolov4-tflite/core/config.py
* __C.YOLO.CLASSES  = "./data/classes/coco.names" 에서 __C.YOLO.CLASSES  = "./data/classes/obj.names" 로 수정
* custom model의 classes name을 적용하기 위함

In [ ]:
# move obj.names to classes 
!mv /content/obj.names /content/tensorflow-yolov4-tflite/data/classes

### 2. convert weight to pb
* weights를 변환하여 pb 를 생성한다
* 지정한 경로에 save_model.py 에 wegihts를 전달하면 output에 지정한 경로에 생성됨

In [ ]:
%cd /content/tensorflow-yolov4-tflite
# save_model.py --wegihts your_weight --output output_path --input_size 정수 --model yolov3 | yolov4 --framework tflite [--tiny]
!python save_model.py --weights ../model.weights --output /content --input_size 416 --model yolov4 --framework tflite 

/content/tensorflow-yolov4-tflite
2020-08-27 08:14:12.281747: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-27 08:14:14.567872: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-27 08:14:14.632160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-27 08:14:14.632962: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-27 08:14:14.633033: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-27 08:14:14.894988: I tensor

### 3. conver pb to tflite
* 2단계에서 생성 된 pb파일로부터 tflite 파일을 생성한다
* --weights 옵션에 pb파일이 위치한 디렉토리의 경로를 전달(pb파일 경로가 아님)

In [ ]:
%cd /content/tensorflow-yolov4-tflite
# python convert_tflite.py --weights pb_path -- output output_path
!python convert_tflite.py --weights /content/ --output /content/model.tflite

/content/tensorflow-yolov4-tflite
2020-08-27 08:17:41.986769: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-27 08:17:43.748675: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-27 08:17:43.766988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-27 08:17:43.767946: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-27 08:17:43.768011: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-27 08:17:43.775100: I tensor

### 4. 드라이브에 저장

In [ ]:
%cd /content
!cp model.tflite /mydrive/model.tflite

/content
